In [1]:
!pip install -Uqq fastai fastbook

get Dataset

In [2]:
from fastai.text.all import *
import torchtext

train_iter, test_iter = torchtext.datasets.IMDB(root='../data', split=('train', 'test'))

train_df = pd.DataFrame(train_iter, columns=['label', 'text'])
test_df = pd.DataFrame(test_iter, columns=['label', 'text'])



100%|██████████| 84.1M/84.1M [00:02<00:00, 38.7MB/s]


WE can augment data in pandas here 

In [78]:
tf = train_df[:1000].append(train_df[24000:25000])
tester = test_df[:1000].append(test_df[24000:25000])

In [79]:
display(tf)

,label,text
0,neg,"I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered ""controversial"" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain politica..."
1,neg,"""I Am Curious: Yellow"" is a risible and pretentious steaming pile. It doesn't matter what one's political views are because this film can hardly be taken seriously on any level. As for the claim that frontal male nudity is an automatic NC-17, that isn't true. I've seen R-rated films with male nudity. Granted, they only offer some fleeting views, but where are the R-rated films with gaping vulvas and flapping labia? Nowhere, because they don't exist. The same goes for those crappy cable shows: schlongs swinging in the breeze but not a clitoris in sight. And those pretentious indie movies li..."
2,neg,"If only to avoid making this type of film in the future. This film is interesting as an experiment but tells no cogent story.<br /><br />One might feel virtuous for sitting thru it because it touches on so many IMPORTANT issues but it does so without any discernable motive. The viewer comes away with no new perspectives (unless one comes up with one while one's mind wanders, as it will invariably do during this pointless film).<br /><br />One might better spend one's time staring out a window at a tree growing.<br /><br />"
3,neg,"This film was probably inspired by Godard's Masculin, féminin and I urge you to see that film instead.<br /><br />The film has two strong elements and those are, (1) the realistic acting (2) the impressive, undeservedly good, photo. Apart from that, what strikes me most is the endless stream of silliness. Lena Nyman has to be most annoying actress in the world. She acts so stupid and with all the nudity in this film,...it's unattractive. Comparing to Godard's film, intellectuality has been replaced with stupidity. Without going too far on this subject, I would say that follows from the dif..."
4,neg,"Oh, brother...after hearing about this ridiculous film for umpteen years all I can think of is that old Peggy Lee song..<br /><br />""Is that all there is??"" ...I was just an early teen when this smoked fish hit the U.S. I was too young to get in the theater (although I did manage to sneak into ""Goodbye Columbus""). Then a screening at a local film museum beckoned - Finally I could see this film, except now I was as old as my parents were when they schlepped to see it!!<br /><br />The ONLY reason this film was not condemned to the anonymous sands of time was because of the obscenity case spa..."
...,...,...
24995,pos,"A hit at the time but now better categorised as an Australian cult film. The humour is broad, unsubtle and, in the final scene where a BBC studio fire is extinguished by urinating on it, crude. Contains just about every cliche about the traditional Australian pilgrimage to 'the old country', and every cliche about those rapacious, stuck up, whinging, Tory Brits. Would be acceptable to the British because of its strong cast of well known actors, and to Australians of that generation, who can 'get' the humour. Americans -- forget it. The language and jokes are in the Australian dialect of En..."
24996,pos,"I love this movie like no other. Another time I will try to explain its virtues to the uninitiated, but for the moment let me quote a few of pieces the remarkable dialogue, which, please remember, is all tongue in cheek. Aussies and Poms will understand, everyone else-well?<br /><br />(title song lyric)""he can sink a beer, he can pick a queer, in his latest double-breasted

load data


In [80]:
train_lm = TextDataLoaders.from_df(tf, text_col='text', is_lm=True)
train_lm.show_batch()

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


,text,text_
0,"xxbos xxmaj valentine is a horrible movie . xxmaj this is what i thought of it : \n\n xxmaj acting : xxmaj very bad . xxmaj katherine xxmaj xxunk can not act . xxmaj the other 's were n't much better . \n\n xxmaj story : xxmaj the story was okay , but it could have been more developed . xxmaj this movie had the potential to be a great movie ,","xxmaj valentine is a horrible movie . xxmaj this is what i thought of it : \n\n xxmaj acting : xxmaj very bad . xxmaj katherine xxmaj xxunk can not act . xxmaj the other 's were n't much better . \n\n xxmaj story : xxmaj the story was okay , but it could have been more developed . xxmaj this movie had the potential to be a great movie , but"
1,"kids wo n't understand some of the darker moments in the film , which makes this film rather watchable for adults . i was also impressed with the cinematography , using animation and digital animation to create a seamless network of pans and xxunk . xxmaj the musical score was once again solid , proving xxmaj hans xxmaj zimmer is the go - to guy when it comes to animated scores ,","wo n't understand some of the darker moments in the film , which makes this film rather watchable for adults . i was also impressed with the cinematography , using animation and digital animation to create a seamless network of pans and xxunk . xxmaj the musical score was once again solid , proving xxmaj hans xxmaj zimmer is the go - to guy when it comes to animated scores , and"
2,"particularly funny scene involves 2 british sisters ( evans and xxmaj shelley ) who seem amused by everything anyone says , but when xxmaj lemmon xxunk out his photos of kids and , yes , ex - wife - to - be , he has the girls xxunk along with him before xxmaj matthau can show up with the promised drinks ! \n\n xxmaj very entertaining . xxbos xxmaj here 's an","funny scene involves 2 british sisters ( evans and xxmaj shelley ) who seem amused by everything anyone says , but when xxmaj lemmon xxunk out his photos of kids and , yes , ex - wife - to - be , he has the girls xxunk along with him before xxmaj matthau can show up with the promised drinks ! \n\n xxmaj very entertaining . xxbos xxmaj here 's an interesting"
3,"as a housewife living in a small town . xxmaj she 's had some experience with local theater and has ambitions of going to xxmaj broadway . xxmaj when a big - time actor arrives in town , she pursues him in hopes that he can give her a career boost . xxmaj but , her husband is worried about shenanigans -- as this actor is a cad . xxmaj so ,","a housewife living in a small town . xxmaj she 's had some experience with local theater and has ambitions of going to xxmaj broadway . xxmaj when a big - time actor arrives in town , she pursues him in hopes that he can give her a career boost . xxmaj but , her husband is worried about shenanigans -- as this actor is a cad . xxmaj so , the"
4,"and xxmaj pumbaa met via their xxunk . xxmaj but there is n't much more than that . \n\n xxmaj it xxunk some scenes from the original , then it ab - xxunk about how this movie changed them a little bit . xxmaj but still , is that it ? i was hoping for something a little more . xxmaj instead , all i have to show for it is an","xxmaj pumbaa met via their xxunk . xxmaj but there is n't much more than that . \n\n xxmaj it xxunk some scenes from the original , then it ab - xxunk about how this movie changed them a little bit . xxmaj but still , is that it ? i was hoping for something a little more . xxmaj instead , all i have to show for it is an empty"
5,"xxup is a good , old - fashioned scary movie … something that 's rather rare nowadays . xxbos xxmaj that reviewers liked this movie surprises me . xxmaj the plot is a xxunk . xxmaj the characters are wooden . xxmaj michael xxmaj xxunk spends most of the film spying on the other characters and xxunk all of them . xxmaj no one has any redeeming quality or point - of","is a good , old - fashioned scary movie … something that 's rather rare n

In [81]:
learn = language_model_learner(train_lm, AWD_LSTM, metrics=[accuracy, Perplexity()], wd=0.1).to_fp16()

In [82]:
import torch
torch.cuda.empty_cache()


In [83]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.386915,4.022849,0.277022,55.860004,01:22


fine tune

In [84]:
learn.unfreeze()
learn.fit_one_cycle(5, 1e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.089097,3.949054,0.284153,51.886272,01:37
1,3.983710,3.893379,0.289465,49.076458,01:37
2,3.855970,3.867990,0.292863,47.846107,01:37
3,3.734733,3.855643,0.293610,47.259007,01:38
4,3.654365,3.856240,0.294063,47.287231,01:38


In [85]:
learn.save_encoder('finetuned')

load text classification model

In [86]:
train_class = TextDataLoaders.from_df(tf, text_col='text', label_col='label' ,text_vocab=train_lm.vocab)
train_class.show_batch()

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


,text,category
0,"xxbos xxmaj in xxup nyc , seaman xxmaj michael o'hara ( orson xxmaj welles ) rescues xxmaj elsa xxmaj bannister ( rita xxmaj hayworth ) from a xxunk & rape as she takes a horse & carriage through xxmaj central xxmaj park -and lives to regret it . xxmaj xxunk - haired xxmaj hayworth 's a xxunk blonde in this one ; as dazzling as fresh - fallen snow -but nowhere near as pure … \n\n xxmaj to reveal any more of the convoluted plot in this xxunk "" noir "" would be criminal . xxmaj it 's as xxunk as the mirrors used to xxunk effect in the final scenes -but the film holds far darker secrets : xxmaj from the xxup ny xxmaj times : "" childhood xxmaj shadows : xxmaj the xxmaj hidden xxmaj story xxmaj of xxmaj the xxmaj black xxmaj dahlia xxmaj murder "" by",pos
1,"xxbos xxmaj by 1987 xxmaj hong xxmaj kong had given the world such films as xxmaj xxunk xxmaj hung 's ` encounters of the xxmaj spooky xxmaj kind ' xxmaj chow xxmaj yun xxmaj fat in xxmaj john xxmaj woo 's iconic ` a xxmaj better xxmaj tomorrow ' , ` zu xxmaj warriors ' and the classic ` mr xxmaj vampire ' . xxmaj jackie xxmaj chan was having international success on video , but it was with ` a xxmaj chinese xxmaj ghost xxmaj story ' that xxup hk cinema had its first real xxunk theatrical hit in the xxmaj west for many years . \n\n xxmaj western xxunk had never seen anything like it . xxmaj it was a film that took various ingredients that xxup hk cinema had used for years ( flying swordsman , wildly choreographed martial arts and the supernatural ) and blended them",pos
2,"xxbos xxmaj well , on it 's credit side ( if it can be said to have one ) , xxmaj timothy xxmaj hines xxup did manage to capture the original setting of xxup h.g . xxmaj wells ' outstanding xxunk . xxmaj but other than that - well , to call a xxunk a xxunk - it sucks xxunk . xxmaj what the xxmaj master xxmaj ed xxmaj wood could have done with the alleged $ 20 million dollar budget ! xxmaj timothy xxmaj hines really does make xxmaj mr . xxmaj wood , who was a flawed genius anyway , look like the best filmmaker of all time . xxmaj the special effects ( i guess you 'd call them that ) are not even up to computer game standards . xxmaj the acting is , well , perhaps about dinner theater xxunk , and the accents are",neg
3,"xxbos xxmaj despite loving xxmaj rita xxmaj hayworth , finding the final few sequences of the film intriguing and being able to appreciate some of the xxunk "" symbolic "" aspects of the cinematography , xxmaj the xxmaj lady from xxmaj shanghai did n't quite work for me . i had a problem with most of the performances , the script and the overall structure . xxmaj and in a film that 's mostly people talking with each other in various situations , that 's quite a problem . xxmaj the xxmaj lady from xxmaj shanghai ended up at a very low "" c "" , or a 7 , for me . \n\n xxmaj the xxmaj lady from xxmaj shanghai is really all about xxmaj orson xxmaj welles ' character , xxmaj michael o'hara . o'hara sees xxmaj elsa "" xxunk "" xxmaj bannister ( rita xxmaj hayworth )",pos
4,"xxbos xxmaj around the late 1970 's , xxunk xxmaj don xxmaj bluth , frustrated with the output his company , xxmaj disney was xxunk , xxunk from the xxmaj mouse xxmaj house to form his own studio . xxmaj his first production , xxup the xxup secret xxup of xxup nimh , was a brilliant feature that still holds up well to this day . xxmaj this was followed by xxup an xxmaj american xxup tail and xxup the xxup land xxup before xxup time , both of which were made under the involvement of xxmaj steven xxmaj spielberg and were xxunk successful . xxmaj although none of those two films had the dark adult appeal of xxup nimh , they still are very charming , enjoyable features for both children and grown - ups . xxmaj but before long , xxmaj don xxmaj bluth had his first major",neg
5,"xxbos xxmaj already his first claim , that desires are always artificial , is totally xxunk . \n\n xxmaj when a xxmaj xxunk xxmaj witness rej

In [87]:
learn = text_classifier_learner(train_class, AWD_LSTM, drop_mult=0.5, metrics=accuracy)
learn = learn.load_encoder('finetuned')

In [88]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.450346,0.428057,0.837500,00:26


In [89]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.328497,0.302270,0.877500,00:39


In [90]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.230268,0.262797,0.885000,00:50
1,0.202747,0.252097,0.887500,00:50


In [91]:
test_dl = learn.dls.test_dl(tester, with_labels=True )


In [92]:
acc = learn.validate(dl=test_dl)[1]
print(acc)

0.8230000138282776
